In [2]:
import numpy as np
from numpy import pi
# importing Qiskit
from qiskit import QuantumCircuit, execute, Aer, IBMQ
from qiskit.providers.ibmq import least_busy
from qiskit.tools.monitor import job_monitor
from qiskit.visualization import plot_histogram, plot_bloch_multivector
from qiskit import QuantumCircuit, execute, Aer
from qiskit.visualization import plot_histogram, plot_bloch_vector
from math import sqrt, pi
%config InlineBackend.figure_format = 'svg' # Makes the images look nice

ここでは、基底状態のエネルギー推定に似た問題を解くために、単純な単一クォビット変分法を使用します。具体的には、ランダムな確率ベクトル x⃗ が与えられ、それが x⃗ に近い確率分布を出力するような単一クォビット変分法のパラメータ化を決定したいと考えています (ここで、近さは2つの確率ベクトル間のマンハッタン距離で定義されます)。

まず、pythonでランダム確率ベクトルを作成します。

In [12]:
import numpy as np
np.random.seed(999999)
target_distr = np.random.rand(2)
# We now convert the random vector into a valid probability vector
target_distr /= sum(target_distr)
print(target_distr)

[0.51357006 0.48642994]


続いて、単一のU3変分形のパラメータを引数にとり、対応する量子回路を返す関数を作成します。

In [4]:
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
def get_var_form(params):
    qr = QuantumRegister(1, name="q")
    cr = ClassicalRegister(1, name='c')
    qc = QuantumCircuit(qr, cr)
    qc.u3(params[0], params[1], params[2], qr[0])
    qc.measure(qr, cr[0])
    return qc

ここでは、変分形式のパラメータのリストを入力として取り、それらのパラメータに関連するコストを返す目的関数を指定します。

In [5]:
from qiskit import Aer, execute
backend = Aer.get_backend("qasm_simulator")
NUM_SHOTS = 10000

def get_probability_distribution(counts):
    output_distr = [v / NUM_SHOTS for v in counts.values()]
    if len(output_distr) == 1:
        output_distr.append(1 - output_distr[0])
    return output_distr

def objective_function(params):
    # Obtain a quantum circuit instance from the paramters
    qc = get_var_form(params)
    # Execute the quantum circuit to obtain the probability distribution associated with the current parameters
    result = execute(qc, backend, shots=NUM_SHOTS).result()
    # Obtain the counts for each measured state, and convert those counts into a probability vector
    output_distr = get_probability_distribution(result.get_counts(qc))
    # Calculate the cost as the distance between the output distribution and the target distribution
    cost = sum([np.abs(output_distr[i] - target_distr[i]) for i in range(2)])
    return cost

最後に、COBYLAオプティマイザのインスタンスを作成し、アルゴリズムを実行します。実行するたびに出力が変化することに注意してください。また、近いうちは、得られた分布が目標分布と完全に一致しないことがありますが、ショット数を増やすことで、出力の精度が上がります。

In [13]:
from qiskit.aqua.components.optimizers import COBYLA

# Initialize the COBYLA optimizer
optimizer = COBYLA(maxiter=500, tol=0.0001)

# Create the initial parameters (noting that our single qubit variational form has 3 parameters)
params = np.random.rand(3)
ret = optimizer.optimize(num_vars=3, objective_function=objective_function, initial_point=params)

# Obtain the output distribution using the final parameters
qc = get_var_form(ret[0])
counts = execute(qc, backend, shots=NUM_SHOTS).result().get_counts(qc)
output_distr = get_probability_distribution(counts)

print("Target Distribution:", target_distr)
print("Obtained Distribution:", output_distr)
print("Output Error (Manhattan Distance):", ret[1])
print("Parameters Found:", ret[0])

[0.71980995 0.08125538 0.65299855]


/home/oryosu/anaconda3/envs/qiskit/lib/python3.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: The QuantumCircuit.u3 method is deprecated as of 0.16.0. It will be removed no earlier than 3 months after the release date. You should use QuantumCircuit.u instead, which acts identically. Alternatively, you can decompose u3 in terms of QuantumCircuit.p and QuantumCircuit.sx: u3(ϴ,φ,λ) = p(φ+π) sx p(ϴ+π) sx p(λ) (2 pulses on hardware).
  


Target Distribution: [0.51357006 0.48642994]
Obtained Distribution: [0.0001, 0.9999]
Output Error (Manhattan Distance): 1.0269401187388392
Parameters Found: [0.71980995 0.08125538 0.65299855]


すでに議論されているように、多項式パラメータ化された変分形は、任意の状態への変換を生成することができません。変分形式は、この制限にどのように対処するかによって、2つのカテゴリに分けることができます。最初のカテゴリの変分形式は、可能な出力状態のセットを制限するために、ドメインまたはアプリケーション固有の知識を使用します。第2のアプローチは、事前のドメインまたはアプリケーション固有の知識なしでヒューリスティック回路を使用します。

第 1 のカテゴリーの変分形式は、必要とされる可能性のある変換のセットを制限するために、問題領域の特性を利用します。例えば、分子の基底状態エネルギーを計算する場合、系内の粒子数は事前に知られている。したがって、正しい粒子数の開始状態が使用されている場合、変分形式を粒子保存変換のみを生成するように制限することで、新しい変換部分空間にまたがるために必要なパラメータの数を大幅に削減することができます。実際、Coupled-Cluster理論からの同様の情報を利用することで、変分形式UCCSDは、Hartree Fock状態から出発したときの分子基底状態のエネルギー推定のために非常に正確な結果を得ることができる。領域固有の知識を利用するもう一つの例として、実際の量子ハードウェア上で実現可能な回路のセットを考えることが挙げられます。超伝導量子ビットをベースにした量子コンピュータは、量子ビットの接続性が限られています。つまり、任意の量子ビットペアに（スワップゲートを挿入せずに）2ビットゲートを実装することはできません。そのため、特定の量子コンピュータアーキテクチャのために、回路が特定の量子デバイスのネイティブに利用可能な接続性とゲートを最大限に利用できるように特別に調整された変分形が構築されてきました。このような変分形は、2017年にIBM量子コンピュータ上でBeH 2ほどの大きさの分子の基底状態エネルギーの推定のためのVQEの実装に成功するために使用されました[4]。

第二のアプローチでは、ゲートは、広い範囲の状態で良好な近似が得られるように層状になっています。Qiskit Aquaは、このような3つの変分形式をサポートしています。RyRz、Ry、SwapRzです（最初の2つについてのみ説明します）。これらの変分形はすべて、ユーザーが指定した複数の設定を受け入れることができます。3つの重要な設定は、系内のクビット数、深さの設定、エンタングルメントの設定です。可変形の1つの層は、シングルクビットの回転とCXゲートの特定のパターンを指定します。深さ設定は、変分形式がこのパターンを何回繰り返すかを指定します。深さの設定を大きくすることで、最適化されなければならないパラメータの数を増やす代償として、変分形が生成できる状態のセットが増えます。最後に，もつれの設定は，CXゲートの構成，そして暗黙のうちに数を選択します．例えば、もつれ設定が線形の場合、CXゲートは隣接するクビットペアに順番に適用されます（したがって、レイヤごとにn-1のCXゲートが追加されます）。絡み合い設定がフルの場合は、各層の各クビットペアにCXゲートが適用されます。entanglement="full "と "linear "に対応するRyRzの回路は、以下のコードスニペットを実行することで確認できます。

In [7]:
from qiskit.circuit.library import EfficientSU2
entanglements = ["linear", "full"]
for entanglement in entanglements:
    form = EfficientSU2(num_qubits=4, entanglement=entanglement)
    if entanglement == "linear":
        print("=============Linear Entanglement:=============")
    else:
        print("=============Full Entanglement:=============")
    # We initialize all parameters to 0 for this demonstration
    display(form.draw(fold=100))
    print()

=============Linear Entanglement:=============


┌──────────┐┌──────────┐     ┌──────────┐┌───────────┐                          ┌───────────┐»
q_0: ┤ RY(θ[0]) ├┤ RZ(θ[4]) ├──■──┤ RY(θ[8]) ├┤ RZ(θ[12]) ├───────────────────■──────┤ RY(θ[16]) ├»
     ├──────────┤├──────────┤┌─┴─┐└──────────┘└┬──────────┤┌───────────┐    ┌─┴─┐    └───────────┘»
q_1: ┤ RY(θ[1]) ├┤ RZ(θ[5]) ├┤ X ├─────■───────┤ RY(θ[9]) ├┤ RZ(θ[13]) ├────┤ X ├──────────■──────»
     ├──────────┤├──────────┤└───┘   ┌─┴─┐     └──────────┘├───────────┤┌───┴───┴───┐    ┌─┴─┐    »
q_2: ┤ RY(θ[2]) ├┤ RZ(θ[6]) ├────────┤ X ├──────────■──────┤ RY(θ[10]) ├┤ RZ(θ[14]) ├────┤ X ├────»
     ├──────────┤├──────────┤        └───┘        ┌─┴─┐    ├───────────┤├───────────┤    └───┘    »
q_3: ┤ RY(θ[3]) ├┤ RZ(θ[7]) ├─────────────────────┤ X ├────┤ RY(θ[11]) ├┤ RZ(θ[15]) ├─────────────»
     └──────────┘└──────────┘                     └───┘    └───────────┘└───────────┘             »
«     ┌───────────┐                          ┌───────────┐┌───────────┐                          
«q_0: ┤ RZ(θ[20]) ├───────────────────■──────┤ RY(θ[24]) ├┤ RZ(θ[28]) ├──────────────────────────
«     ├───────────┤┌───────────┐    ┌─┴─┐    └───────────┘├───────────┤┌───────────┐             
«q_1: ┤ RY(θ[17]) ├┤ RZ(θ[21]) ├────┤ X ├──────────■──────┤ RY(θ[25]) ├┤ RZ(θ[29]) ├─────────────
«     └───────────┘├───────────┤┌───┴───┴───┐    ┌─┴─┐    └───────────┘├───────────┤┌───────────┐
«q_2: ──────■──────┤ RY(θ[18]) ├┤ RZ(θ[22]) ├────┤ X ├──────────■──────┤ RY(θ[26]) ├┤ RZ(θ[30]) ├
«         ┌─┴─┐    ├───────────┤├───────────┤    └───┘        ┌─┴─┐    ├───────────┤├───────────┤
«q_3: ────┤ X ├────┤ RY(θ[19]) ├┤ RZ(θ[23]) ├─────────────────┤ X ├────┤ RY(θ[27]) ├┤ RZ(θ[31]) ├
«         └───┘    └───────────┘└───────────┘                 └───┘    └───────────┘└───────────┘


=============Full Entanglement:=============


┌──────────┐┌──────────┐               ┌──────────┐┌───────────┐                         »
q_0: ┤ RY(θ[0]) ├┤ RZ(θ[4]) ├──■────■────■──┤ RY(θ[8]) ├┤ RZ(θ[12]) ├─────────────────────────»
     ├──────────┤├──────────┤┌─┴─┐  │    │  └──────────┘└───────────┘┌──────────┐┌───────────┐»
q_1: ┤ RY(θ[1]) ├┤ RZ(θ[5]) ├┤ X ├──┼────┼───────■────────────■──────┤ RY(θ[9]) ├┤ RZ(θ[13]) ├»
     ├──────────┤├──────────┤└───┘┌─┴─┐  │     ┌─┴─┐          │      └──────────┘├───────────┤»
q_2: ┤ RY(θ[2]) ├┤ RZ(θ[6]) ├─────┤ X ├──┼─────┤ X ├──────────┼───────────■──────┤ RY(θ[10]) ├»
     ├──────────┤├──────────┤     └───┘┌─┴─┐   └───┘        ┌─┴─┐       ┌─┴─┐    ├───────────┤»
q_3: ┤ RY(θ[3]) ├┤ RZ(θ[7]) ├──────────┤ X ├────────────────┤ X ├───────┤ X ├────┤ RY(θ[11]) ├»
     └──────────┘└──────────┘          └───┘                └───┘       └───┘    └───────────┘»
«                            ┌───────────┐┌───────────┐                                            »
«q_0: ──────■────────■────■──┤ RY(θ[16]) ├┤ RZ(θ[20]) ├────────────────────────────────■────────■──»
«         ┌─┴─┐      │    │  └───────────┘└───────────┘┌───────────┐┌───────────┐    ┌─┴─┐      │  »
«q_1: ────┤ X ├──────┼────┼────────■────────────■──────┤ RY(θ[17]) ├┤ RZ(θ[21]) ├────┤ X ├──────┼──»
«     ┌───┴───┴───┐┌─┴─┐  │      ┌─┴─┐          │      └───────────┘├───────────┤┌───┴───┴───┐┌─┴─┐»
«q_2: ┤ RZ(θ[14]) ├┤ X ├──┼──────┤ X ├──────────┼────────────■──────┤ RY(θ[18]) ├┤ RZ(θ[22]) ├┤ X ├»
«     ├───────────┤└───┘┌─┴─┐    └───┘        ┌─┴─┐        ┌─┴─┐    ├───────────┤├───────────┤└───┘»
«q_3: ┤ RZ(θ[15]) ├─────┤ X ├─────────────────┤ X ├────────┤ X ├────┤ RY(θ[19]) ├┤ RZ(θ[23]) ├─────»
«     └───────────┘     └───┘                 └───┘        └───┘    └───────────┘└───────────┘     »
«          ┌───────────┐┌───────────┐                                       
«q_0: ──■──┤ RY(θ[24]) ├┤ RZ(θ[28]) ├───────────────────────────────────────
«       │  └───────────┘└───────────┘┌───────────┐┌───────────┐             
«q_1: ──┼────────■────────────■──────┤ RY(θ[25]) ├┤ RZ(θ[29]) ├─────────────
«       │      ┌─┴─┐          │      └───────────┘├───────────┤┌───────────┐
«q_2: ──┼──────┤ X ├──────────┼────────────■──────┤ RY(θ[26]) ├┤ RZ(θ[30]) ├
«     ┌─┴─┐    └───┘        ┌─┴─┐        ┌─┴─┐    ├───────────┤├───────────┤
«q_3: ┤ X ├─────────────────┤ X ├────────┤ X ├────┤ RY(θ[27]) ├┤ RZ(θ[31]) ├
«     └───┘                 └───┘        └───┘    └───────────┘└───────────┘

このセクションでは、プログラム的アプローチを用いたVQEの実装を説明します。Qiskit Aquaは宣言的な実装も可能にしますが、基礎となるアルゴリズムに関する情報が少なくなります。このコード、特にクビット演算子の準備は、Qiskit Tutorialsのリポジトリにあるコードに基づいています（2019年7月現在、https://github.com/Qiskit/qiskit-tutorials ）。

まず、以下のライブラリをインポートする必要があります。

In [9]:
from qiskit.aqua.algorithms import VQE, NumPyEigensolver
import matplotlib.pyplot as plt
import numpy as np
from qiskit.chemistry.components.variational_forms import UCCSD
from qiskit.chemistry.components.initial_states import HartreeFock
from qiskit.circuit.library import EfficientSU2
from qiskit.aqua.components.optimizers import COBYLA, SPSA, SLSQP
from qiskit.aqua.operators import Z2Symmetries
from qiskit import IBMQ, BasicAer, Aer
from qiskit.chemistry.drivers import PySCFDriver, UnitsType
from qiskit.chemistry import FermionicOperator
from qiskit import IBMQ
from qiskit.aqua import QuantumInstance
from qiskit.ignis.mitigation.measurement import CompleteMeasFitter
from qiskit.providers.aer.noise import NoiseModel

様々な原子間距離におけるLiHの基底状態エネルギーの計算を実証した。各原子間距離で分子のドライバーを作らなければならない。この実験では、使用するqubitsの数を減らすために、コアを凍結させ、2つの未占有軌道を除去していることに注意してください。まず、原子間距離を取り、適切なクォビット演算子 H と、その演算子に関する他の情報を返す関数を定義する。

In [10]:
def get_qubit_op(dist):
    driver = PySCFDriver(atom="Li .0 .0 .0; H .0 .0 " + str(dist), unit=UnitsType.ANGSTROM, 
                         charge=0, spin=0, basis='sto3g')
    molecule = driver.run()
    freeze_list = [0]
    remove_list = [-3, -2]
    repulsion_energy = molecule.nuclear_repulsion_energy
    num_particles = molecule.num_alpha + molecule.num_beta
    num_spin_orbitals = molecule.num_orbitals * 2
    remove_list = [x % molecule.num_orbitals for x in remove_list]
    freeze_list = [x % molecule.num_orbitals for x in freeze_list]
    remove_list = [x - len(freeze_list) for x in remove_list]
    remove_list += [x + molecule.num_orbitals - len(freeze_list)  for x in remove_list]
    freeze_list += [x + molecule.num_orbitals for x in freeze_list]
    ferOp = FermionicOperator(h1=molecule.one_body_integrals, h2=molecule.two_body_integrals)
    ferOp, energy_shift = ferOp.fermion_mode_freezing(freeze_list)
    num_spin_orbitals -= len(freeze_list)
    num_particles -= len(freeze_list)
    ferOp = ferOp.fermion_mode_elimination(remove_list)
    num_spin_orbitals -= len(remove_list)
    qubitOp = ferOp.mapping(map_type='parity', threshold=0.00000001)
    qubitOp = Z2Symmetries.two_qubit_reduction(qubitOp, num_particles)
    shift = energy_shift + repulsion_energy
    return qubitOp, num_particles, num_spin_orbitals, shift

最初に、クビット演算子と古典的な正確な固有値ソルバーを使用して、正確な基底状態のエネルギーが計算されます。その後、初期状態｜ψ⟩が生成され、これをVQEインスタンスが最終的なアンサッツminθ(|ψ(θ)⟩)を生成するために使用します。各原子間距離における正確な結果とVQEの結果が格納されます。vqe.run(backend)['energy'] + shiftで与えられた結果は、minθ(⟨ψ(θ)|H|ψ(θ)⟩)と等価であり、最小値は必ずしもグローバル最小値ではないことに注意してください。

VQE(qubitOp, var_form, optimizer, 'matrix')でVQEインスタンスを初期化する場合、Hの期待値｜ψ(θ)⟩上のHは行列の乗算によって直接計算されます。しかし、実際の量子デバイスや、VQE(qubitOp, var_form, optimizer, 'paulis')を用いたqasm_simulatorのような真のシミュレータを使用する場合、期待値の計算はより複雑になります。ハミルトニアンはパウリ文字列の和として表現され、各パウリ項は使用されるマッピングで指定された通りにクォビットに作用します。それぞれのパウリ文字列は、対応する回路が｜ψ(θ)⟩ に対応する回路に付加されています。その後、これらの回路がそれぞれ実行され、すべての結果が｜ψ(θ)⟩ の H の期待値を決定するために使用されます。次の例では、VQE インスタンスを行列モードで初期化しているので、期待値は行列の乗算によって直接計算されます。

以下のコードスニペットは、実行して完了するまでに数分かかる場合があることに注意してください。

In [14]:
backend = BasicAer.get_backend("statevector_simulator")
distances = np.arange(0.5, 4.0, 0.1)
exact_energies = []
vqe_energies = []
optimizer = SLSQP(maxiter=5)
for dist in distances:
    qubitOp, num_particles, num_spin_orbitals, shift = get_qubit_op(dist)
    result = NumPyEigensolver(qubitOp).run()
    exact_energies.append(np.real(result.eigenvalues) + shift)
    initial_state = HartreeFock(
        num_spin_orbitals,
        num_particles,
        qubit_mapping='parity'
    ) 
    var_form = UCCSD(
        num_orbitals=num_spin_orbitals,
        num_particles=num_particles,
        initial_state=initial_state,
        qubit_mapping='parity'
    )
    print(var_form)
    vqe = VQE(qubitOp, var_form, optimizer)
    vqe_result = np.real(vqe.run(backend)['eigenvalue'] + shift)
    vqe_energies.append(vqe_result)
    print("Interatomic Distance:", np.round(dist, 2), "VQE Result:", vqe_result, "Exact Energy:", exact_energies[-1])
    
print("All energies have been calculated")

Interatomic Distance: 0.5 VQE Result: -7.039673441021999 Exact Energy: [-7.03973252]
Interatomic Distance: 0.6 VQE Result: -7.313308880249155 Exact Energy: [-7.31334583]
Interatomic Distance: 0.7 VQE Result: -7.500887034731441 Exact Energy: [-7.50092209]
Interatomic Distance: 0.8 VQE Result: -7.630945409722851 Exact Energy: [-7.63097825]
Interatomic Distance: 0.9 VQE Result: -7.720781098910903 Exact Energy: [-7.72081241]
Interatomic Distance: 1.0 VQE Result: -7.782211599433442 Exact Energy: [-7.7822424]
Interatomic Distance: 1.1 VQE Result: -7.823568080962408 Exact Energy: [-7.82359928]
Interatomic Distance: 1.2 VQE Result: -7.850666132356581 Exact Energy: [-7.85069838]
Interatomic Distance: 1.3 VQE Result: -7.867529586531959 Exact Energy: [-7.86756329]
Interatomic Distance: 1.4 VQE Result: -7.8769662545341 Exact Energy: [-7.87700149]
Interatomic Distance: 1.5 VQE Result: -7.880979507885269 Exact Energy: [-7.88101572]
Interatomic Distance: 1.6 VQE Result: -7.881036616636475 Exact Energ